## TensorBoardサンプルコード

* tensorflow==1.13.1

on_epoch_endイベント時にTensorBoardに画像を表示する。

### 簡易なモデルを記述（fashion-mnist）

In [2]:
import datetime
import numpy as np

import tensorflow as tf
from tensorflow import keras
from sklearn import model_selection
from tensorflow.keras import layers

# データの用意
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# testデータの1/10をvalidationデータに使う
test_images, val_images, test_labels, val_labels = model_selection.train_test_split(
    test_images, test_labels, test_size=0.1, stratify=test_labels)

# 単純な推定器
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

## ラベル一覧

In [3]:
LABELS = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
]

### 利用可能なフォント一覧を出力するコード

In [ ]:
import matplotlib.font_manager
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

### 画像を出力する

on_epoch_end で画像を出力してみる。

画像の変換については下記を参照した。

https://github.com/tensorflow/models/blob/v1.13.0/research/object_detection/utils/visualization_utils.py#L80

In [4]:
from  PIL import Image, ImageDraw, ImageFont
import six

# フォントの指定
font = ImageFont.truetype("NotoSansCJK-Bold.ttc", 24)

class MyCallback(tf.keras.callbacks.TensorBoard):
    
    def __init__(self, val_images, val_labels, **kwargs):
        super().__init__(**kwargs)
        self.val_images = val_images
        self.val_labels = val_labels
    
    def encode_image_array_as_png_str(self, image, pred, label):
        # 画像をPillowのImageに変換
        # 画像に文字を書込む為にサイズを増やしてRGBに
        image = Image.fromarray(np.uint8(image)) \
            .resize((256, 256)).convert('RGB')
        # 画面下部に赤文字でpred, labelを表示
        message = "P=%s\nL=%s" % (LABELS[pred], LABELS[label])
        draw = ImageDraw.Draw(image)
        draw.text((20, 180), message, (255, 0, 0), font=font)
        # Valueに指定できるよう形式を変換
        with six.BytesIO() as output:
            image.save(output, format='PNG')
            png_string = output.getvalue()
        return png_string

    def on_epoch_end(self, epoch, logs={}):
        super().on_epoch_end(epoch, logs)

        # predict実行
        y_pred = np.argmax(model.predict(test_images), axis=1).astype(np.int8)
        
        # 画像の設定
        random_index = np.random.randint(0, len(test_labels), size=10)
        def get_image_value(i):
            label = test_labels[i]
            pred = y_pred[i]
            image = test_images[i] * 255
            image_str = self.encode_image_array_as_png_str(image, label, pred)
            img_summary = tf.Summary.Image(encoded_image_string=image_str)
            return tf.Summary.Value(tag='image/%d' % i, image=img_summary)
        img_values = [get_image_value(i) for i in random_index]

        summary = tf.Summary(value=img_values)
        self.writer.add_summary(summary, epoch)
        self.writer.flush()

In [4]:
my_callback = MyCallback(
    val_images, val_labels,
    log_dir='logs')

model.fit(
    train_images,
    train_labels,
    epochs=10,
    callbacks=[my_callback])

Epoch 1/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.5033 - acc: 0.8228
Epoch 2/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3794 - acc: 0.8629
Epoch 3/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.3402 - acc: 0.8766
Epoch 4/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.3161 - acc: 0.8847
Epoch 5/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.2997 - acc: 0.8900
Epoch 6/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.2828 - acc: 0.8955
Epoch 7/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.2690 - acc: 0.8997
Epoch 8/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.2589 - acc: 0.9040
Epoch 9/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.2489 - acc: 0.9071
Epoch 10/10
60000/60000 [==============================] - 5s 78us/sample - loss: 